<a href="https://colab.research.google.com/github/vinaykumartadepall/Word2Vec/blob/master/Word2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
import nltk
from nltk.tokenize import RegexpTokenizer, sent_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
word_tokenizer = RegexpTokenizer(r'\w+')
import ast
import pickle
import numpy as np
import matplotlib.pyplot as plt
import random
import glob
import os.path as op
import re

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
raw_data = []
for line in open('/content/drive/My Drive/a1/reviews_Electronics_5.json', 'r'):
raw_data.append(json.loads(line)['reviewText'].lower())

raw_data = np.array(raw_data[:1000])
# print(raw_data[0])

In [ ]:
def pre_process(raw_data):
    word_indices = {}
    word_frequency = {}
    ind = 0
    total_size = 0
    for para in raw_data:
        tokens = tokenizer.tokenize(para)
        for token in tokens:
            if not token in word_frequency:
                word_frequency[token] = 1
            else:
                word_frequency[token]+=1

    for word in word_frequency:
        # if word_frequency[word] < 5:
        #     del word_frequency[word]
        if (word_frequency[word] > 5) and (word not in stopwords.words('english')) and not (bool(re.search(r'\d', word))):
            total_size += word_frequency[word]
            word_indices[word] = ind
            ind+=1
    
    print(total_size)
    
    new_data = []
    new_size = 0
    for para in raw_data:
        new_para = []
        for sentence in sent_tokenize(para):
            new_sentence = []
            tokens = tokenizer.tokenize(sentence)
            for token in tokens:
                if token in word_indices:
                    ratio = word_frequency[token]/total_size
                    prob = min(1.0,(np.sqrt(ratio/0.001)+1)*(0.001/ratio))
                    prob = 1
                    randn = np.random.randint(0,1000000)/1000000
                    if randn <= prob:
                        new_size+=1
                        new_sentence.append(token)
            new_para.append(new_sentence)
        new_data.append(new_para)

    # print(np.sum(np.array([len(para) for para in new_data])))
    print(new_size)

    return new_data, word_indices, ind

new_data, word_indices, vocab_size = pre_process(raw_data)

word_indices['$'] = vocab_size
with open('/content/drive/My Drive/a1/word_indices.json', 'w') as outfile:
    json.dump(word_indices, outfile)
    
with open('/content/drive/My Drive/a1/new_data.json', 'wb') as outfile:
    pickle.dump(new_data,outfile)


62674
62674


In [ ]:
def load_wordIndices():
    with open('/content/drive/My Drive/a1/word_indices.json', 'r') as infile:
        word_indices = json.load(infile)

    with open('/content/drive/My Drive/a1/new_data.json', 'rb') as infile:
        data = pickle.load(infile)

    return data, word_indices, word_indices['$']
    
data, word_indices, vocab_size= load_wordIndices()
vocab_size = 0
for word in word_indices:
    if word_indices[word]>=0:
            vocab_size += 1

frequencies = {}

for para in data:
    for sentence in para:
        for word in sentence:
            if word_indices[word] not in frequencies:
                frequencies[word_indices[word]] = 0
            frequencies[word_indices[word]]+=1

index_word = {}
for i in word_indices:
    index_word[word_indices[i]] = i

print("data size without subsampling = ", np.sum(np.array([len(para) for para in data])))
print("Vocabulary size = ", vocab_size)
# print(data)
# print(vocab_size)
# print(data[0])

data size without subsampling =  7315
Vocabulary size =  1840


In [ ]:
def generate_train_data(data, window_size):
    train_data = []
    for para in data:
        for sentence in para:
            for i in range(len(sentence)):
                sample = [sentence[i]]
                context = []
                for j in range(i-window_size, i+window_size+1):
                    if j>=0 and j<len(sentence) and j!=i:
                        context.append(sentence[j])
                sample.append(context)
                if len(context) != 0:
                    train_data.append(sample)
    return train_data

train_data = generate_train_data(data, window_size=2)
print("No of training samples generated = ",len(train_data))

simws = []
for sample in train_data:
    if sample[0] == "camera":
        simws.append(sample)

print(len(simws))
print(simws)

# random.shuffle(train_data)

No of training samples generated =  62421
87
[['camera', ['consider', 'negative']], ['camera', ['tablet', 'acer', 'actually', 'use']], ['camera', ['screen', 'front', 'camera', 'matter']], ['camera', ['front', 'camera', 'matter']], ['camera', ['friends', 'tablets', 'never', 'used']], ['camera', ['store', 'computer']], ['camera', ['lacks', 'gps', 'bluetooth']], ['camera', ['still', 'gps']], ['camera', ['tablet', 'except', 'gps', 'personally']], ['camera', ['front', 'nook', 'phone']], ['camera', ['nook', 'tablet']], ['camera', ['bought', 'knew', 'advance', 'one']], ['camera', ['always', 'takes']], ['camera', ['important', 'real']], ['camera', ['files', 'need', 'buy']], ['camera', ['say', 'though', 'nook', 'much']], ['camera', ['much', 'better', 'new', 'one']], ['camera', ['reviews', 'guess', 'angles', 'made']], ['camera', ['microphone', 'apps', 'place']], ['camera', ['need', 'phone', 'ipod', 'dvd']], ['camera', ['care', 'gps']], ['camera', ['lack', 'microphone', 'total', 'cost']], ['camer

In [ ]:
print(train_data[:100])

[['got', ['gps', 'husband']], ['gps', ['got', 'husband', 'road']], ['husband', ['got', 'gps', 'road']], ['road', ['gps', 'husband']], ['impressed', ['shipping', 'time']], ['shipping', ['impressed', 'time', 'arrived']], ['time', ['impressed', 'shipping', 'arrived', 'days']], ['arrived', ['shipping', 'time', 'days', 'earlier']], ['days', ['time', 'arrived', 'earlier', 'expected']], ['earlier', ['arrived', 'days', 'expected', 'within']], ['expected', ['days', 'earlier', 'within', 'week']], ['within', ['earlier', 'expected', 'week', 'use']], ['week', ['expected', 'within', 'use', 'however']], ['use', ['within', 'week', 'however', 'started']], ['however', ['week', 'use', 'started', 'could']], ['started', ['use', 'however', 'could', 'unit']], ['could', ['however', 'started', 'unit']], ['unit', ['started', 'could']], ['worked', ['great', 'worked']], ['great', ['worked', 'worked']], ['worked', ['worked', 'great']], ['work', ['great', 'normal']], ['great', ['work', 'normal', 'person']], ['norma

In [ ]:
# train_data = [["hi",["hello","welcome"]], ["bye", ["hello", "welcome"]], ["cool",["bell","bill"]], ["hot", ["bell", "bill"]]]
# vocab_size = 8
# word_indices = {"hi":0, "hello":1, "welcome":2, "bye":3, "cool":4, "bell":5, "bill":6, "hot":7}
# wordvec_dim = 5
# w1 = np.random.rand(vocab_size,wordvec_dim)
# w2 = np.random.rand(vocab_size, wordvec_dim)

In [ ]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

def softmax(x):
    sum = np.sum(np.exp(x))
    # print(sum)
    return (np.exp(x)/sum)

# def softmax_loss(target_idx, h, w1, w2):
#     u = np.matmul(w2,h)
#     y = softmax(u)
#     return -np.log(y[target_idx])

def softmax_loss_grad(target_idx, x, w1, w2, context_ids):
    h = np.matmul(w1.T,x)
    # print(h)
    y = softmax(np.matmul(w2,h))
    loss = -np.log(y[target_idx])
    
    y[target_idx] -= 1
    
    # grad_w1 = np.zeros(w1.shape)
    back_weights = np.matmul(w2.T,y)
    if np.isnan(loss):
        print("NaN encountered!!!")
        return 0, np.zeros(w1.shape), np.zeros(w2.shape)
    # for id in context_ids:
    #     grad_w1[id] = back_weights
    
    # grad_w1 /= len(context_ids)
    # return loss, grad_w1, np.outer(h,y).T
    return loss, np.outer(x, back_weights), np.outer(h, y).T

def get_negSamples(num_negSamples):
    samples = []
    for i in range(num_negSamples):
        rint = random.randint(0,vocab_size-1)
        samples.append(rint)
    return samples



def negSampling_loss_grad(target_idx, context_idx, w1, w2):
    negSamples = get_negSamples(num_negSamples = 10)
    grad_w1 = np.zeros(w1.shape)
    grad_w2 = np.zeros(w2[0].shape)
    loss = -np.log(sigmoid(np.dot(w1[context_idx],w2[target_idx])))
    loss -= np.sum(np.array([np.log(sigmoid(-np.dot(w1[x],w2[target_idx]))) for x in negSamples]))
    
    h = w1[context_idx]
    y = sigmoid(np.dot(h,w2[target_idx]))
    grad_w2 += h*(y-1)
    grad_w1[context_idx] += w2[target_idx]*(y-1)

    for i in negSamples:
        h1 = w1[i]
        y = sigmoid(-np.dot(h1, w2[target_idx]))
        grad_w2 -= h1*(y-1)
        grad_w1[target_idx] -= w2[target_idx]*(y-1)
    
    return loss, grad_w1,grad_w2

In [ ]:
# def cbow_loss(context, target, w1, w2, word_indices):
#     h = np.zeros(len(w1[0]))
#     for word in context:
#         h += w1[word_indices[word]]
#     h/=len(context)
#     return softmax_loss(word_indices[target], h, w1, w2)

def cbow_loss_grad(context, target, w1, w2, word_indices):
    h = np.zeros(len(w1[0]))
    inp = np.zeros(vocab_size)
    context_ids = []
    for word in context:
        inp[word_indices[word]] = 1
        context_ids.append(word_indices[word])
        h += w1[word_indices[word]]
    inp /= len(context)
    h /= len(context)
    return softmax_loss_grad(word_indices[target], inp, w1, w2, context_ids)


def skipgram_loss_grad(context, target, w1, w2, word_indices):
    loss = 0.0
    grad_w1 = np.zeros(w1.shape)
    grad_w2 = np.zeros(w2.shape)
    for word in context:
        loss_here, w1_here, w2_here = negSampling_loss_grad(word_indices[target], word_indices[word], w1, w2)
        loss += loss_here
        grad_w1 += w1_here
        grad_w2[word_indices[target]] += w2_here
    
    return loss, grad_w1, grad_w2
        

In [ ]:
def objective(w1, w2, word_indices, train_data, model, start, batch_size=500):
    loss = 0
    grad_w1 = np.zeros(w1.shape)
    grad_w2 = np.zeros(w2.shape)
    batch_size = min(batch_size , len(train_data)-start)
    for i in range(batch_size):
        sample = train_data[start+i]
        loss_part, grad_w1_part, grad_w2_part = model(sample[1], sample[0], w1, w2, word_indices)
        loss += loss_part
        grad_w1 += grad_w1_part
        grad_w2 += grad_w2_part

    return loss, grad_w1/batch_size, grad_w2/batch_size

In [ ]:
def load_saved_params(fname):
    with open(fname,'r') as f:
        params = json.load(f)
        return params

def save_params(iter, params):
    params_file = "/content/drive/My Drive/a1/saved_params_%d.npy" % iter
    np.save(params_file, params)

def most_similar(tar_word, w2):
    tar_idx = word_indices[tar_word]
    sim = []
    for i in range(vocab_size):
        word = index_word[i]
        sim.append([np.dot(w2[tar_idx],w2[word_indices[word]]) / (np.linalg.norm(w2[tar_idx]) * np.linalg.norm(w2[word_indices[word]])),word])
    matches = sorted(sim,reverse=True)
    return matches[:10]
    # print(matches)
# print(index_word)


In [ ]:
def train(word_indices, train_data, wordvec_dim, batch_size, epochs = 500, notify_per = 10, learning_dec = 100, save_every = 200, learning_rate = 0.5):
    w1 = np.random.rand(vocab_size,wordvec_dim)
    w2 = np.random.rand(vocab_size, wordvec_dim)
    rp = 0
    # rp,weights,state = load_saved_params()
    # if len(weights[0]) != 0:
    #     w1 = weights[0]
    #     w2 = weights[1]
    #     learning_rate *= (0.8)** rp
    for i in range(max(rp,1),epochs):
        st = 0
        loss = 0
        while st < len(train_data):
            loss_here, w1_grad, w2_grad = objective(w1,w2,word_indices, train_data,model=skipgram_loss_grad, start = st)
            w1 -= w1_grad*learning_rate
            w2 -= w2_grad*learning_rate
            loss += loss_here
            st += batch_size
            if st//batch_size % notify_per == 0:
                print("Batch ",st/batch_size," completed : loss = ", loss, " Learning rate = ", learning_rate)
                loss = 0
            if st//batch_size % 20 == 0:
                save_params(st//batch_size,np.array([w1,w2]))
        
        print("------ Epoch ",i," : loss = ", loss)
        save_params(i,np.array([w1,w2]))
        learning_rate *= 1
        sims = most_similar("camera",w2)
        print("10 most words similar to camera")
        for k in sims:
            print(k)
    
    return w1,w2
    
w1,w2 = train(word_indices, train_data, wordvec_dim = 50, batch_size=500, epochs = 5000)
w1_t,w2_t = w1,w2

In [ ]:
# visualizeWords = [
#     "great", "cool", "speak", "wonderful", "well", "amazing",
#     "worth", "bad",
#     "annoying", "man",
#      "coffee"]

ret = load_saved_params('/content/drive/My Drive/NLP_a1_embeddings/w1_embeddings_CBOW.json')
w2 = [ret[word] for word in ret]

visualizeWords  = [ "hi", "hello", "welcome", "bye","cool","bell","bill","hot"]

visualizeIdx = [word_indices[word] for word in visualizeWords]
visualizeVecs = w2[visualizeIdx, :]
temp = (visualizeVecs - np.mean(visualizeVecs, axis=0))
covariance = 1.0 / len(visualizeIdx) * temp.T.dot(temp)
U,S,V = np.linalg.svd(covariance)
coord = temp.dot(U[:,0:2])
print(coord)
for i in range(len(visualizeWords)):
    plt.text(coord[i,0], coord[i,1], visualizeWords[i],
        bbox=dict(facecolor='green', alpha=0.1))

plt.xlim((np.min(coord[:,0]), np.max(coord[:,0])))
plt.ylim((np.min(coord[:,1]), np.max(coord[:,1])))

plt.show()

index_word = {}
for i in word_indices:
    index_word[word_indices[i]] = i

def most_similar(tar_word):
    tar_idx = word_indices[tar_word]
    sim = []
    for i in range(vocab_size):
        word = index_word[i]
        sim.append([np.dot(w2[tar_idx],w2[word_indices[word]]) / (np.linalg.norm(w2[tar_idx]) * np.linalg.norm(w2[word_indices[word]])),word])
    matches = sorted(sim,reverse=True)
    return matches[1:10]
    # print(matches)
# print(index_word)
sims = most_similar("hi")
print("10 most words similar to camera")
for i in sims:
    print(i)


KeyError: ignored